<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **日常支出速算與分攤（作業一）**
目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。

AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。

Sheet 欄位: date, category, item, amount, payer

GoogleSheet: https://docs.google.com/spreadsheets/d/18VYb4uiN1XspCMFks6KPOj2Js5UIt1icceR78dedn28/edit?usp=sharing


In [11]:
# 套件、認證
import pandas as pd
import numpy as np
import gspread
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)



In [12]:
# 資料
SHEET_URL = "https://docs.google.com/spreadsheets/d/18VYb4uiN1XspCMFks6KPOj2Js5UIt1icceR78dedn28/edit"
RECORDS_SHEET = "Records"
SUMMARY_SHEET = "Summary"

def read_records_df():
    ws = client.open_by_url(SHEET_URL).worksheet(RECORDS_SHEET)
    data = ws.get_all_values()
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame(data[1:], columns=data[0])

    if "amount" in df.columns:
        df["amount"] = pd.to_numeric(df["amount"], errors="coerce").fillna(0)
    return df


In [13]:
# 總支出、分類小計、AA 分攤、個人支出
def calc_summary(df):
    total_spent = df["amount"].sum()

    # 分類小計
    category_summary = df.groupby("category")["amount"].sum().to_dict()

    # AA 分攤
    aa_df = df[df["type"]=="AA"]
    payers = aa_df["payer"].unique()
    n = len(payers) if len(payers) > 0 else 1
    aa_share = aa_df["amount"].sum() / n
    balances = {}
    for payer in payers:
        paid = aa_df[aa_df["payer"]==payer]["amount"].sum()
        balances[payer] = round(paid - aa_share, 2)

    # RZ個人支出
    personal_df = df[(df["type"]=="個人") & (df["payer"]=="RZ")]

    return total_spent, category_summary, balances, personal_df


In [14]:

# Summary
def update_summary_sheet():
    df = read_records_df()
    if df.empty:
        print("Records 無資料，無法更新 Summary")
        return None

    total_spent, category_summary, balances, personal_df = calc_summary(df)

    sh = client.open_by_url(SHEET_URL)
    try:
        ws_summary = sh.worksheet(SUMMARY_SHEET)
        ws_summary.clear()
    except gspread.WorksheetNotFound:
        ws_summary = sh.add_worksheet(title=SUMMARY_SHEET, rows=100, cols=10)

    # DataFrame
    rows = []
    rows.append(["總支出", total_spent, "", "", ""])
    rows.append(["--- 分類小計 ---", "", "", "", ""])
    for cat, amt in category_summary.items():
        rows.append([cat, amt, "", "", ""])
    rows.append(["--- 分攤結果 ---", "", "", "", ""])
    for payer, bal in balances.items():
        rows.append([payer, bal, "", "", ""])

    # 顯示個人支出
    if not personal_df.empty:
        rows.append(["--- 個人支出（RZ） ---", "", "", "", ""])
        for _, r in personal_df.iterrows():
            rows.append([r["category"], r["item"], r["amount"], r["payer"], r["type"]])

    df_summary = pd.DataFrame(rows, columns=["項目1","項目2","項目3","項目4","項目5"])

    df_summary_clean = df_summary.applymap(
        lambda x: int(x) if isinstance(x, (np.integer))
                  else float(x) if isinstance(x, (np.floating))
                  else str(x)
    )

    # 寫回 Google Sheet
    ws_summary.update(
        values=[df_summary_clean.columns.tolist()] + df_summary_clean.values.tolist(),
        range_name="A1"
    )

    print("Summary 已更新")
    return df_summary_clean

df_summary = update_summary_sheet()
df_summary


/tmp/ipython-input-67702221.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_summary_clean = df_summary.applymap(


Summary 已更新


,項目1,項目2,項目3,項目4,項目5
0,總支出,5750,,,
1,--- 分類小計 ---,,,,
2,entertainment,1100,,,
3,food,1940,,,
4,other,300,,,
5,shopping,2000,,,
6,traffic,410,,,
7,--- 分攤結果 ---,,,,
8,RZ,-398.33,,,
9,WY,136.67,,,


In [15]:
#Gradio
import gradio as gr
import datetime

def gradio_update_summary(selected_person):
    """更新 Summary 分頁，並回傳 DataFrame"""
    df_summary = update_summary_sheet(person=selected_person if selected_person != "全部" else None)
    return df_summary

with gr.Blocks(title="日常支出速算與分攤") as demo:
    gr.Markdown("## 🧾 日常支出速算與分攤(Gradio)")

    # 新資料進入 Summary
    with gr.Tab("更新 Summary"):
        person_dropdown = gr.Dropdown(
            label="選擇個人支出顯示人",
            choices=["RZ","WY","Alice","全部"],
            value="RZ"
        )
        summary_out = gr.Dataframe(label="Summary", interactive=False)
        update_btn = gr.Button("更新 Google Sheet Summary")
        update_btn.click(fn=gradio_update_summary, inputs=[person_dropdown], outputs=[summary_out])

    # 支出
    with gr.Tab("新增支出"):
        date_in = gr.Textbox(label="日期 YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
        category_in = gr.Textbox(label="分類")
        item_in = gr.Textbox(label="品項")
        amount_in = gr.Number(label="金額")
        payer_in = gr.Textbox(label="付款人")
        type_in = gr.Dropdown(label="類型", choices=["AA","個人"], value="AA")
        add_btn = gr.Button("新增到 Records")
        add_msg = gr.Markdown()

        def gr_add_expense(date, category, item, amount, payer, typ):
            add_expense(date, category, item, amount, payer, typ)
            return f"✅ 已新增 {date} {category} {item} {amount} {payer} {typ}"

        add_btn.click(
            fn=gr_add_expense,
            inputs=[date_in, category_in, item_in, amount_in, payer_in, type_in],
            outputs=[add_msg]
        )

# 啟動 Gradio 介面
demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>